In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['abcnews-date-text.csv']


In [38]:
import string
import nltk
#nltk.download('stopwords')
#nltk.download('vader_lexicon')
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from matplotlib import pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation

In [39]:
news=pd.read_csv('../input/abcnews-date-text.csv')

In [40]:
news.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [41]:
news.publish_date.min(),news.publish_date.max()

(20030219, 20171231)

In [42]:
news['headline_text']=news['headline_text'].str.lower().str.replace('[^a-z# ]','')

In [43]:
stop=stopwords.words('english')

In [44]:
def sw(text):
    text=[word for word in text.split() if word not in stop and len(word)>3]
    return " ".join(text)

In [45]:
doc_clean_final=news['headline_text'].apply(sw)

In [46]:
# Creating the tfidf vectorizer and DTM

tfidf_vec = TfidfVectorizer()

score = tfidf_vec.fit_transform(doc_clean_final)


In [50]:
# Build LDA model with sklearn

lda_model = LatentDirichletAllocation(n_topics=10, max_iter=15,random_state=1234)

lda_output = lda_model.fit_transform(score)


print(lda_model)  # Model attributes

/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=15, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=10, perp_tol=0.1,
             random_state=1234, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [54]:
topicnames = ["Topic " + str(i) for i in range(lda_model.n_topics)]

# How to see the dominant topic in each document?

# Create the column names of Document - topic matrix


topicnames = ["Topic " + str(i) for i in range(lda_model.n_topics)]

# Create the index names of Document - topic matrix

docnames = ["Doc " + str(i) for i in range(len(doc_clean_final))]

# Make the pandas dataframe with cells containg the probablities of each document to clasify under each topic

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)


# Get dominant topic for each document

dominant_topic = np.argmax(df_document_topic.values, axis=1)

df_document_topic['dominant_topic'] = dominant_topic

#df_document_topic

docnames = ["Doc " + str(i) for i in range(len(doc_clean_final))]

# Make the pandas dataframe with cells containg the probablities of each document to clasify under each topic

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document

dominant_topic = np.argmax(df_document_topic.values, axis=1)

df_document_topic['dominant_topic'] = dominant_topic

df_document_topic.head()

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,dominant_topic
Doc 0,0.03,0.25,0.03,0.27,0.24,0.03,0.03,0.03,0.03,0.03,3
Doc 1,0.03,0.03,0.39,0.36,0.03,0.03,0.03,0.03,0.03,0.03,2
Doc 2,0.03,0.03,0.03,0.70,0.03,0.03,0.03,0.03,0.03,0.03,3
Doc 3,0.03,0.03,0.03,0.03,0.53,0.03,0.03,0.03,0.20,0.03,4
Doc 4,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.70,9


In [55]:
# Review topics distribution across documents

df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")

df_topic_distribution.columns = ['Topic Num', 'Num Documents']

df_topic_distribution.head()

,Topic Num,Num Documents
0,3,193407
1,4,142429
2,2,138265
3,8,115918
4,7,108304


In [56]:
# visualize the LDA model with pyLDAvis

import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda_model, score, tfidf_vec, mds='tsne')

panel

/opt/conda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3       51.158226 -103.169281       1        1  14.524802
4       82.051628  348.930328       2        1  11.691452
2       65.689476  110.060349       3        1  11.620850
8     -181.235626   19.794319       4        1  10.385537
7       34.922958 -342.045898       5        1   9.764020
1      274.665466 -245.006256       6        1   9.395813
9     -157.711960  251.911423       7        1   8.973152
5      298.025513  -12.902884       8        1   8.471016
6     -189.438553 -213.364166       9        1   8.153154
0      306.383698  220.219574      10        1   7.020205, topic_info=      Category         Freq       Term        Total  loglift  logprob
term                                                                 
59964  Default  8917.000000     police  8917.000000  30.0000  30.0000
17702  Default  2959.000000    country  2959.000000  29.0000  29.0000
17963  Default  3978.000000      crash  3978.000000  28.0000  28.0000
13728  Default  3424.000000    charged  3424.000000  27.0000  27.0000
53621  Default  2676.000000       news  2676.000000  26.0000  26.0000
36523  Default  2464.000000       hour  2464.000000  25.0000  25.0000
67479  Default  3168.000000      rural  3168.000000  24.0000  24.0000
52214  Default  2912.000000     murder  2912.000000  23.0000  23.0000
17736  Default  4520.000000      court  4520.000000  22.0000  22.0000
47666  Default  2525.000000     market  2525.000000  21.0000  21.0000
41965  Default  3008.000000     killed  3008.000000  20.0000  20.0000
15134  Default  1740.000000     closer  1740.000000  19.0000  19.0000
86473  Default  2724.000000      woman  2724.000000  18.0000  18.0000
13731  Default  2031.000000    charges  2031.000000  17.0000  17.0000
38879  Default  5355.000000  interview  5355.000000  16.0000  16.0000
50574  Default  2676.000000    missing  2676.000000  15.0000  15.0000
39159  Default  2079.000000       iraq  2079.000000  14.0000  14.0000
17646  Default  4418.000000    council  4418.000000  13.0000  13.0000
500    Default  2554.000000    accused  2554.000000  12.0000  12.0000
66331  Default  2090.000000       rise  2090.000000  11.0000  11.0000
33173  Default  1667.000000     guilty  1667.000000  10.0000  10.0000
4194   Default  1631.000000    assault  1631.000000   9.0000   9.0000
59834  Default  1296.000000    podcast  1296.000000   8.0000   8.0000
34724  Default  3034.000000     health  3034.000000   7.0000   7.0000
32215  Default  4595.000000       govt  4595.000000   6.0000   6.0000
73664  Default  1271.000000      sport  1271.000000   5.0000   5.0000
39620  Default  1819.000000     jailed  1819.000000   4.0000   4.0000
24323  Default  2379.000000   election  2379.000000   3.0000   3.0000
43235  Default  2045.000000      labor  2045.000000   2.0000   2.0000
19710  Default  3441.000000      death  3441.000000   1.0000   1.0000
...        ...          ...        ...          ...      ...      ...
85031  Topic10   812.738831  wednesday   813.609504   2.6553  -5.3038
80690  Topic10   781.731365    tuesday   782.602040   2.6553  -5.3427
69911  Topic10   775.226066  september   776.096748   2.6553  -5.3511
78607  Topic10   762.612299   thursday   763.482972   2.6552  -5.3675
4624   Topic10   754.253546     august   755.124223   2.6552  -5.3785
55047  Topic10   725.623435    october   726.494114   2.6552  -5.4172
54447  Topic10   705.782140   november   706.652820   2.6551  -5.4449
40608  Topic10   685.627010       july   686.497692   2.6551  -5.4739
40635  Topic10   678.478243       june   679.348926   2.6551  -5.4844
62454  Topic10   717.239105    quarter   718.164056   2.6551  -5.4288
3423   Topic10   643.547688      april   644.418371   2.6550  -5.5372
2611   Topic10   613.758780   analysis   614.629466   2.6550  -5.5846
19820  Topic10   591.075245   december   591.945925   2.6549  -5.6223
9956   Topic10   517.521743  breakfast   

In [57]:
# Get the top 10 keywords each topic so that we can make some comment

# Show top n keywords for each topic

def show_topics(vectorizer= tfidf_vec, model=lda_model, n_words=10):
    keywords = np.array(tfidf_vec.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer= tfidf_vec, model= lda_model, n_words=15)        

# Creating Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)

# Creating the columns names and index names
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]

df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,country,news,hour,rural,podcast,sport,drum,grandstand,business,march,friday,national,monday,wednesday,tuesday
Topic 1,killed,iraq,bomb,kills,troops,attack,blast,dead,soldiers,israel,afghan,kill,pakistan,police,afghanistan
Topic 2,police,charged,court,murder,death,woman,missing,accused,charges,found,jailed,guilty,assault,drug,search
Topic 3,council,govt,plan,health,water,funding,urged,boost,group,funds,indigenous,centre,plans,mine,call
Topic 4,election,labor,says,minister,govt,rudd,opposition,abbott,government,union,talks,chief,calls,defends,deal
Topic 5,crash,dies,plane,killed,accident,train,injured,truck,highway,dead,fire,road,rescue,crashes,interview
Topic 6,closer,tigers,rain,toll,blues,port,storm,warriors,clash,australia,bulls,eagles,back,bulldogs,sharks
Topic 7,interview,world,final,hill,wins,test,open,weather,england,australia,tour,broken,round,murray,ashes
Topic 8,market,rise,prices,rate,price,farmers,rates,share,high,australian,dollar,profit,shares,record,dairy
Topic 9,fire,asylum,firefighters,boat,island,blaze,wild,north,bushfire,warning,fires,illegal,crews,bird,korea


In [58]:
# Adding the column of topic to our data frame

news['topic'] = dominant_topic

news.head()



,publish_date,headline_text,topic
0,20030219,aba decides against community broadcasting lic...,3
1,20030219,act fire witnesses must be aware of defamation,2
2,20030219,a g calls for infrastructure protection summit,3
3,20030219,air nz staff in aust strike for pay rise,4
4,20030219,air nz strike to affect australian travellers,9
